In [1]:
import numpy as np
import pandas as pd
import pycountry

In [63]:
data = pd.read_csv("WHO-COVID-19-global-data.csv")
data.head(5)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [3]:
def cc(x):
    if(x == 'XA' or x == 'XC' or x == 'XB'):
        code = 'BES'
    elif(x=='XK'):
        code = 'XKX'
    elif(len(x)==3):
        return x
    elif(len(x)<=1):
        return 'OTH'
    else:
        code = pycountry.countries.get(alpha_2 = x).alpha_3
    return code

In [4]:
data.Country_code[data.Country_code.isna()] = 'NAM'

C:\Users\SameenShaukat\AppData\Local\Temp\ipykernel_14952\3968862160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Country_code[data.Country_code.isna()] = 'NAM'


In [5]:
data['CountryID'] = data['Country_code'].apply(cc)

In [6]:
sum_data = data[["Country", "CountryID","New_cases","New_deaths"]].copy()

In [7]:
def cc_num(x):
    if(x == 'XKX'):
        code = '383'
    elif(x=='OTH'):
        code = 'OTH'
    else:
        code = pycountry.countries.get(alpha_3 = x).numeric
    return code

In [8]:
sum_data['CountryIDNum'] = sum_data['CountryID'].apply(cc_num)

In [9]:
sum_data = sum_data.groupby(["Country","CountryIDNum"]).sum().reset_index().copy()

In [10]:
sum_data = sum_data[sum_data.CountryIDNum != 'OTH'].copy()

In [11]:
sum_data.head(5)

,Country,CountryIDNum,New_cases,New_deaths
0,Afghanistan,004,185272,7743
1,Albania,008,304600,3527
2,Algeria,012,267194,6876
3,American Samoa,016,7471,33
4,Andorra,020,45508,153


In [12]:
sum_data.to_csv("Cummulative_Cases_CountryWise.csv", index=False)

In [54]:
Top10CountriesStat = sum_data.nlargest(10, 'New_cases').reset_index(drop=True).copy()

In [55]:
Top10CountriesStat.to_csv('Top10CountriesStat.csv',index=False)

In [60]:
Top10CountriesStat.Country[Top10CountriesStat.Country=='Türkiye'] = 'Turkey'

C:\Users\SameenShaukat\AppData\Local\Temp\ipykernel_14952\1069791791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Top10CountriesStat.Country[Top10CountriesStat.Country=='Türkiye'] = 'Turkey'


In [62]:
Top10CountriesStat[Top10CountriesStat.Country=='Turkey']

,Country,CountryIDNum,New_cases,New_deaths
9,Turkey,792,15524071,99184


In [64]:
data.head(2)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0


In [101]:
data['date'] = pd.to_datetime(data['Date_reported'])
data['year'], data['month'] ,data['monthname']= data['date'].dt.year, data['date'].dt.month, data['date'].dt.strftime("%b")

In [102]:
data.Country[data.Country=='Türkiye']='Turkey'

C:\Users\SameenShaukat\AppData\Local\Temp\ipykernel_14952\2445573446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Country[data.Country=='Türkiye']='Turkey'


In [103]:
ydata = data[["Country","year","New_cases","New_deaths"]].copy()
mdata = data[["Country","year","month","monthname","New_cases","New_deaths"]].copy()

In [104]:
ydata = ydata.groupby(["Country","year"]).sum().reset_index().copy()
mdata = mdata.groupby(["Country","year","month","monthname"]).sum().reset_index().copy()

In [105]:
cntry = Top10CountriesStat.Country.to_list()

In [106]:
cntry

['United States of America',
 'India',
 'Brazil',
 'France',
 'Germany',
 'The United Kingdom',
 'Italy',
 'Republic of Korea',
 'Russian Federation',
 'Turkey']

In [107]:
df = ydata[ydata.Country.isin(cntry)].copy()
mdf = mdata[mdata.Country.isin(cntry)].copy()

In [108]:
df.head(15)

,Country,year,New_cases,New_deaths
84,Brazil,2020,7563551,192681
85,Brazil,2021,14700283,426136
86,Brazil,2022,11440559,58987
216,France,2020,2564972,64004
217,France,2021,6917610,56958
218,France,2022,23285870,27500
234,Germany,2020,1734998,50323
235,Germany,2021,5431876,67628
236,Germany,2022,23620435,25904
288,India,2020,10266674,148738


In [109]:
df.to_csv("top10CountriesYearly.csv", index=False)

In [112]:
mdf = mdf.sort_values(by=["Country","year","month"]).copy()

In [113]:
mdf.to_csv("top10CountriesMonthly.csv", index=False)